In [11]:
import os.path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchmetrics.functional import mean_absolute_error


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse_sklearn
from sklearn.metrics import mean_absolute_error as mae_sklearn

from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.hyperopt import HyperOptSearch
from ray.air.integrations.mlflow import MLflowLoggerCallback
from ray.air import session

import mlflow
from mlflow.tracking import MlflowClient

In [12]:
in_dim = 1
hidden_dim = 64
out_dim = 1
sequence_length = 72

In [13]:
def get_data(path):
    data = []
    for i in ["/train", "/val", "/test"]:
        data.append(pd.read_csv(path + i + ".csv", index_col="Datetime"))
    return data[0], data[1], data[2]
# Load data
path = './data/clean_data/univariate_Q_Kalltveit'
train, val, test = get_data(path)

In [14]:
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x), np.array(y)

In [15]:
X_train, y_train = sliding_windows(train.values, sequence_length)
X_val, y_val = sliding_windows(val.values, sequence_length)
X_test, y_test = sliding_windows(test.values, sequence_length)

X_train, X_val, X_test = X_train.reshape(len(X_train), -1), X_val.reshape(len(X_val), -1), X_test.reshape(len(X_test), -1)
print(X_train.shape, X_val.shape, X_test.shape)

(44228, 72) (13217, 72) (5624, 72)


In [375]:
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize(mean=(0,0,0),std=(1,1,1))])

In [376]:
from torch.utils.data import Dataset
import torch

class data_trans(Dataset):
        
    def __init__(self, data, groundtruth, transform=None):

        self.data = self._get_data(data)
        self.groundtruth = self._get_data(groundtruth)
        self.transform = transform

    def _get_data(self,data):
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self,index):
               
        inputs = self.data[index,:]
        groundtruths = self.groundtruth[index,:]
        
        if self.transform:
                        
            inputs = torch.from_numpy(inputs).float()
            groundtruths = torch.from_numpy(groundtruths).float()
        return [inputs, groundtruths]

In [377]:
train_data_trans = data_trans(X_train, y_train, transform)
val_data_trans = data_trans(X_val, y_val, transform)
test_data_trans = data_trans(X_test, y_test, transform)

In [378]:
class FCN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, sequence_length):

        super(FCN, self).__init__()
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim
        self.sequence_length = sequence_length

        self.layer_in = nn.Linear(sequence_length, hidden_dim, bias=False)
        self.fcn = nn.Linear(hidden_dim, hidden_dim)
        self.layer_out = nn.Linear(hidden_dim, out_dim, bias=False)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward (self, input):
        out = self.layer_in(input)
        out = self.sigmoid(out)
        out = self.fcn(out)
        out = self.sigmoid(out)
        out = self.layer_out(out)
      
        return out

In [379]:
class LSTM(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, sequence_length, mode):
        super(LSTM, self).__init__()
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim
        self.sequence_length = sequence_length
        self.mode = mode
        
        self.lstm1 = nn.LSTMCell(in_dim, hidden_dim)
        self.lstm2 = nn.LSTMCell(hidden_dim, hidden_dim)
        
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.linear_out = nn.Linear(hidden_dim*sequence_length, out_dim)

        
    def forward(self, y):
        outputs = []
        h_t = torch.zeros(y.size(0), self.hidden_dim).to(self.mode["device"])
        c_t = torch.zeros(y.size(0), self.hidden_dim).to(self.mode["device"])
        h_t2 = torch.zeros(y.size(0), self.hidden_dim).to(self.mode["device"])
        c_t2 = torch.zeros(y.size(0), self.hidden_dim).to(self.mode["device"])
        
        for time_step in y.split(1, dim=1):
            h_t, c_t = self.lstm1(time_step, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs.append(output)
        outputs = torch.cat(outputs, dim=1)
        out = self.linear_out(outputs)
        return out

In [380]:
import sys

client = MlflowClient()
cwd = os.getcwd()
exp_base_name = "LSTM"

In [381]:
created = 0
for i in range(100):
    try:
        exp_name = exp_base_name+"_{}".format(i)
        experiment_id = client.create_experiment(exp_name)
        created=1
        break
    except (TypeError, mlflow.exceptions.MlflowException):
        continue

if not created:
    print("ERROR: Try new experiment name.")
    sys.exit(1)

weights_root = "./model_weights/"
weights_dir = weights_root+exp_name+'/'
os.mkdir(weights_dir)

In [382]:
from tqdm import tqdm

def fit(model, loss_function, optimizer, data_loader, num_epochs, mode, use_amp=False):
	history = {"train": {"loss": [], "mae": []}, "val": {"loss": [], "mae": []}}
	scaler = torch.cuda.amp.GradScaler(enabled=use_amp) # Mixed-precision support for compatible GPUs
	print("\nTraining the model:")
	for epoch in range(num_epochs):
		print("\nEpoch", epoch+1)
		if epoch < num_epochs - 1:
			keys = ["train", "val"]
		else:
			keys = ["train", "val", "test"]
		for key in keys:
			dataset_size = 0
			dataset_loss = 0.0
			if key == "train":
				model.train()
			else:
				model.eval()
			for X_batch, y_batch in tqdm(data_loader[key]):
				X_batch, y_batch = X_batch.to(mode["device"]), y_batch.to(mode["device"])
				with torch.set_grad_enabled(mode=(key=="train")): # Autograd activated only during training
					with torch.cuda.amp.autocast(enabled=use_amp): # Mixed-precision support for compatible GPUs
						batch_output = model(X_batch.float())
						batch_loss = loss_function(batch_output, y_batch)
					if key == "train":
						scaler.scale(batch_loss).backward()
						scaler.step(optimizer) 	
						scaler.update()
						optimizer.zero_grad()
				dataset_size += y_batch.shape[0]
				dataset_loss += y_batch.shape[0] * batch_loss.item()
			dataset_loss /= dataset_size
			if key in ["train", "val"]:
				history[key]["loss"].append(dataset_loss)
				if key == "train":
					tune.report(train_loss=dataset_loss)
				else:
					tune.report(val_loss=dataset_loss)
			else:
				print("\nEvaluating the model:")
			print(key, "loss:", dataset_loss)
			tune.report(test_loss=dataset_loss)
	return model


In [384]:
def train_model(config):
    use_GPU = torch.cuda.is_available()
    if use_GPU:
        mode = {"name": "cuda", "device": torch.device("cuda")}
    else:
        mode = {"name": "cpu", "device": torch.device("cpu")}

    model = FCN(in_dim, hidden_dim, out_dim, sequence_length)
    #model = LSTM(in_dim, hidden_dim, out_dim, sequence_length, mode)
    #model = LSTM_A(in_dim, hidden_dim, out_dim, sequence_length, mode)
    model.to(mode["device"])

    loss_function = nn.MSELoss().to(mode["device"])
    optimizer = optim.Adam(model.parameters(), lr=config["lr"]) # 

    train_dataloader = torch.utils.data.DataLoader(train_data_trans,
                                           batch_size = config['batch_size'],
                                           shuffle = True)
    val_dataloader = torch.utils.data.DataLoader(val_data_trans,
                                            batch_size = config['batch_size'],
                                            shuffle = False)
    test_dataloader = torch.utils.data.DataLoader(test_data_trans,
                                            batch_size = config['batch_size'],
                                            shuffle = False)
    data_loader = {
    "train": train_dataloader,
    "val": val_dataloader,
    "test": test_dataloader,
    }
                                           
    num_epochs = 50
    best_trained_model = fit(model, loss_function, optimizer, data_loader, num_epochs, mode)
    out_name = ""
    for k, v in config.items():
        if not k in ['weights_dir', 'cwd']:
            out_name += '{}-{}_'.format(k, v)
    torch.save(best_trained_model.state_dict(), os.path.join(config['cwd'], config['weights_dir'], out_name[:-1] + '.pth'))


In [385]:
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

config = {
    "mlflow_experiment_id": experiment_id,
    "weights_dir": weights_dir,
    "cwd": cwd,
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([128, 128*2, 128*3, 128*4])
}

analysis = tune.run(
    train_model,
    config=config,
    resources_per_trial={"cpu": 12, "gpu": 1},
    num_samples=10,
    callbacks=[MLflowLoggerCallback(experiment_name=exp_name)],
)
